<a href="https://colab.research.google.com/github/hardik0/Multi-Object-Tracking-Google-Colab/blob/main/DEFT-Detection-Embeddings-for-Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEFT: Detection Embeddings for Tracking

<img src="https://github.com/MedChaabane/DEFT/raw/main/DEFT_Diagram.png" height=300px>

**Abstract**

In this paper, we propose an efficient joint detection and tracking model named DEFT, or Detection Embeddings for Tracking. Our approach relies on an appearance-based object matching network jointly-learned with an underlying object detection network. An LSTM is also added to capture motion constraints. DEFT has comparable accuracy and speed to the top methods on 2D online tracking leaderboards while having significant advantages in robustness when applied to more challenging tracking data. DEFT raises the bar on the nuScenes monocular 3D tracking challenge, more than doubling the performance of the previous top method.

**Original code :** [DEFT: Detection Embeddings for Tracking](https://github.com/MedChaabane/DEFT) by
[Mohamed Chaabane](https://github.com/MedChaabane), Peter Zhang, J. Ross Beveridge, Stephen O'Hara
```
@article{Chaabane2021deft,
  title={DEFT: Detection Embeddings for Tracking},
  author={Chaabane, Mohamed and Zhang, Peter and Beveridge, Ross and O'Hara, Stephen},
  journal={arXiv preprint arXiv:2102.02267},
  year={2021}
}
```

In [ ]:
! nvidia-smi
! /usr/local/cuda/bin/nvcc --version

## Installation
Clone this repo, and run the following commands.

In [ ]:
! git clone https://github.com/MedChaabane/DEFT
%cd DEFT

Install PyTorch and the dependencies.

In [ ]:
! pip install torch==1.2.0 torchvision==0.4.0    # CUDA 10.0
# Install COCOAPI:
! pip install cython; pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
! pip install -r requirements.txt  

Restart the runtime 

In [ ]:
%cd DEFT/

/content/DEFT


Compile [DCNv2](https://github.com/CharlesShang/DCNv2)

In [ ]:
%%shell

cd src/lib/model/networks/
git clone https://github.com/CharlesShang/DCNv2
cd DCNv2
./make.sh

Download the [pretrained models](https://drive.google.com/drive/folders/1dlVoV-4fMYlttdj2ba0unn6WX-nxaC48?usp=sharing) and move them to src/models/ .

In [ ]:
#! pip install gdown
%mkdir src/models
! gdown --id 1H7RX3M7wUA55RxeZDkloby9foGgYlhcp -O src/models/model_mot.pth
#! gdown --id 1WFSaZmnT2-xacjf7L9nkudmbShERFzX_ -O src/models/model_kitti.pth

## Datsets Preparation

**MOT 2017**
* Run the dataset preprocessing script.

In [ ]:
# update get_mot_17.sh - unzip -q MOT17.zip
%%shell

mkdir data
cd src/tools/
sh get_mot_17.sh

The output data structure should be:
```
    ${DEFT_ROOT}
  |-- data
  `-- |-- mot17
      `-- |--- train
          |   |--- MOT17-02-FRCNN
          |   |    |--- img1
          |   |    |--- gt
          |   |    |   |--- gt.txt
          |   |    |--- det
          |   |    |   |--- det.txt
          |   |--- ...
          |--- test
          |   |--- MOT17-01-FRCNN
          |---|--- ...
          `---| annotations
              |--- train.json
              `--- test.json
```


## Training

In [ ]:
cd src

/content/DEFT/src


In [ ]:
# edit train.py 
#import warnings
#warnings.filterwarnings("ignore", category=UserWarning)

! python train.py tracking --exp_id mot17_train --dataset mot --dataset_version 17trainval --pre_hm --ltrb_amodal --same_aug --hm_disturb 0.05 --lost_disturb 0.4 --fp_disturb 0.1 --gpus 0 --load_model models/model_mot.pth

Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [4]
Namespace(AFE=True, K=100, add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=4, chunk_sizes=[4], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/content/DEFT/src/lib/../../data', dataset='mot', dataset_version='17trainval', debug=0, debug_dir='/content/DEFT/src/lib/../../exp/tracking/mot17_train/debug', debugger_theme='white', dense_reg=1, dep_weight=1, depth_scale=1, dim_weight=1, dla_node='dcn', down_ratio=4, efficient_level=0, eval_val=False, exp_dir='/content/DEFT/src/lib/../../exp/tracking', exp_id='mot17_train', fix_res=True, fix_short=-1, flip=0.5, flip_test=False, fp_disturb=0.1, gpus=[0], gpus_str='0', head_conv={'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256], 'ltrb_amodal': [256]}, head_kernel=3, heads={'hm': 1, 'reg': 2, 'wh': 2, 'tracking': 2, 'ltrb_amodal': 4}, hm_disturb=0.05, hm_hp_weight=1, 

In [ ]:
%%shell 

cd src
# train
python train.py tracking --exp_id mot17_train --dataset mot --dataset_version 17trainval --pre_hm --ltrb_amodal --same_aug --hm_disturb 0.05 --lost_disturb 0.4 --fp_disturb 0.1 --gpus 0 --load_model models/model_mot.pth
# train the motion model
python train_prediction.py tracking --exp_id mot17_motion_model --dataset mot --dataset_version 17trainval --gpus 0

## Test & Visualization 

In [ ]:
%%shell

cd src
# test
python test.py tracking --exp_id mot17_train --dataset mot --dataset_version 17test --pre_hm --ltrb_amodal --track_thresh 0.4 --pre_thresh 0.5 --load_model models/model_mot.pth

## Demo - To-Do list;